In [62]:
from random import seed
from random import randrange
from csv import reader
from math import sqrt
 
# Tải data từ file csv
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset
 
# Chuyển các cột dữ liệu từ kiểu xâu sang kiểu float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
 
# Chia dũ liệu thành k folds 
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    # Tính số fold 
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            # Chọn ngẫu nhiên từng điểm dữ liệu để đưa vào mỗi fold 
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
 
# Tính accuracy 
def accuracy(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0
 
# Đánh giá thuật toán dựa trên sử dụng cross validation
def evaluate_algorithm(dataset, random_fr, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = random_fr(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy(actual, predicted)
        scores.append(accuracy)
    return scores

# phân nhánh cho cây dựa trên các thuộc tính 
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right
 
# Tính độ sạch gini
def gini_index(groups, classes):
    # Tinhs tổng số lượng mẫu ở mỗi nhánh phân chia
    n_instances = float(sum([len(group) for group in groups]))
    # Tính tổng có trọng số gini cho mỗi nhóm 
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # Tránh việc chia cho 0
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / n_instances)
    return gini
 
# Tìm điểm phân chia tốt nhất cho dữ liệu
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    # Chọn ngẫu nhiên một vài thuộc tính cho mỗi cây
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)
    # Với các thuộc tính chọn được, tiến hành tạo cây dựa trên độ sạch gini 
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}
 
# Tạo nốt cuối(terminal node)
# Sẽ có 4 nốt cuối tương ứng 4 class chia thành các group các điểm dữ liệu, ta sẽ lấy loại class chiếm đa số
# làm nhãn cho nốt đó (vì trong các group sẽ có nhiễu các class khác)
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)
 
# Tách các nhánh con 
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    # Nếu không có phân chia thì đã đến nốt cuối
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # Kiểm tra đạt đến tối đa chiều sâu 
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # Xử lý nốt lá trái 
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
    # Xử lý nốt là phải
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)
 
# Dựng cây quyết định 
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root
 
# Dự đoán với 1 cây quyết định
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']
 
# Tạo một mẫu ngẫu nhiên từ dữ liệu 
def subdata(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

# Dự đoán bằng việc sử dụng phương pháp bag of trees 
def bagging(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)
 
# Dựng rừng ngẫu nhiêu 
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for i in range(n_trees):
        sample = subdata(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
    predictions = [bagging(trees, row) for row in test]
    return(predictions)
 


In [63]:
from random import seed
import pandas as pd


seed(2)
# tải dữ liệu lên từ file csv
filename = 'train.csv'
dataset = load_csv(filename)
# Bỏ đi dòng đầu là tên các đặc trưng
dataset = dataset[1:]
train = []

# Thêm vào các đặc trưng là sc_h*sc_w, px_height*px_width
for i in dataset:
    i.insert(0, str(int(i[11])*int(i[12])))
    i.insert(0, str(int(i[15])*int(i[16])))
    train.append(i)
# Chuyển các kiểu xâu thành kiểu số
for i in range(0, len(dataset[0])-1):
    str_column_to_float(dataset, i)
#str_column_to_int(dataset, len(dataset[0])-1)



In [49]:
print(train[:5])

[[63.0, 15120.0, 842.0, 0.0, 2.2, 0.0, 1.0, 0.0, 7.0, 0.6, 188.0, 2.0, 2.0, 20.0, 756.0, 2549.0, 9.0, 7.0, 19.0, 0.0, 0.0, 1.0, '1'], [51.0, 1799140.0, 1021.0, 1.0, 0.5, 1.0, 0.0, 1.0, 53.0, 0.7, 136.0, 3.0, 6.0, 905.0, 1988.0, 2631.0, 17.0, 3.0, 7.0, 1.0, 1.0, 0.0, '2'], [22.0, 2167308.0, 563.0, 1.0, 0.5, 1.0, 2.0, 1.0, 41.0, 0.9, 145.0, 5.0, 6.0, 1263.0, 1716.0, 2603.0, 11.0, 2.0, 9.0, 1.0, 1.0, 0.0, '2'], [128.0, 2171776.0, 615.0, 1.0, 2.5, 0.0, 0.0, 0.0, 10.0, 0.8, 131.0, 6.0, 9.0, 1216.0, 1786.0, 2769.0, 16.0, 8.0, 11.0, 1.0, 0.0, 0.0, '2'], [16.0, 1464096.0, 1821.0, 1.0, 1.2, 0.0, 13.0, 1.0, 44.0, 0.6, 141.0, 2.0, 14.0, 1208.0, 1212.0, 1411.0, 8.0, 2.0, 15.0, 1.0, 1.0, 0.0, '1']]


In [27]:

# Đánh giá thuật toán bằng việc thử nghiệm các tham số khác nhau để tìm bộ tham số tốt nhất
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = 12
#n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1, 5, 10]:
	scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
	print('Trees: %d' % n_trees)
	print('Scores: %s' % scores)
	print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Trees: 1
Scores: [78.5, 79.75, 79.25, 78.0, 79.75]
Mean Accuracy: 79.050%
Trees: 5
Scores: [88.25, 85.0, 84.75, 85.75, 85.0]
Mean Accuracy: 85.750%
Trees: 10
Scores: [88.75, 89.25, 90.5, 89.0, 88.75]
Mean Accuracy: 89.250%


In [64]:
#Chia tập train ra thành 2 tập train và test để đánh gía mô hình cuối cùng
trainSet = train[:1700]
testSet = train[1700:]

In [65]:
pred = random_forest(trainSet, testSet, max_depth=10, min_size=1, sample_size=1, n_trees=10, n_features=12)

In [66]:
actual = [i[22] for i in testSet]

Độ chính xác đạt 89.66%

In [67]:
print(accuracy(actual, pred))

89.66666666666666
